In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulat

In [5]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [6]:
# 25 EV sessions per weekday
events_25_ev = get_synth_events(25)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Checking session details
session_copy = deepcopy(events_25_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

650 session_11 650 757 13.412234567982312 Plugin
651 session_19 651 760 8.765099205047308 Plugin
654 session_10 654 757 7.767310033352643 Plugin
659 session_3 659 701 14.330349739054252 Plugin
656 session_7 656 776 9.098113407873287 Plugin
656 session_24 656 783 17.415880352320013 Plugin
658 session_14 658 681 2.5821694409678706 Plugin
667 session_15 667 768 5.507366499194126 Plugin
659 session_18 659 743 11.277158055676011 Plugin
658 session_9 658 790 5.3673028785793155 Plugin
659 session_2 659 776 29.844914505572028 Plugin
669 session_1 669 789 46.79114648441875 Plugin
678 session_12 678 802 28.593596716703168 Plugin
673 session_6 673 759 42.123672628656024 Plugin
665 session_13 665 779 17.370294794660218 Plugin
720 session_0 720 848 14.856943312020977 Plugin
674 session_16 674 787 7.522679436606037 Plugin
679 session_17 679 788 9.509265284732766 Plugin
666 session_8 666 793 4.033629261826606 Plugin
674 session_4 674 759 12.502171476891329 Plugin
661 session_20 661 757 9.998102490190

#### Algorithms and new equation

In [8]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty, 1.2),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

#### Run Simulation with New Equation

In [9]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2024, 8, 21))
    end = timezone.localize(datetime(2024, 8, 21))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_25_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_1", "session_16", "session_3", "session_4", "session_17"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [10]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.6040112972259521
('RR', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 2.3027946949005127
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 16.63716197013855
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.5211348533630371
('RR', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 2.298130989074707
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 19.493271112442017
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.5552036762237549
('RR', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 2.7047667503356934
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 15.739328384399414
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.49402570724487305
('RR', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 2.3631885051727295
('MPC', 75)
60.24

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [11]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [12]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [13]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,165.000,35.268,361.684,99.000,0.312
1,three_phase,RR,60,73.119,64.0,103.058,40.628,264.461,61.835,0.372
2,three_phase,MPC,60,72.407,68.0,99.258,65.203,261.885,59.555,0.315
3,three_phase,Unctrl,65,100.000,100.0,152.308,35.268,361.684,99.000,0.313
4,three_phase,RR,65,73.685,68.0,103.431,42.436,266.507,67.230,0.388
5,three_phase,MPC,65,72.271,64.0,99.285,65.042,261.392,64.535,0.366
6,three_phase,Unctrl,70,100.000,100.0,141.429,35.268,361.684,99.000,0.329
7,three_phase,RR,70,73.237,68.0,103.750,38.584,264.888,72.625,0.365
8,three_phase,MPC,70,72.232,44.0,98.129,64.679,261.253,68.690,0.333
9,three_phase,Unctrl,75,100.000,100.0,132.000,35.268,361.684,99.000,0.304


#### Simulation with different algorithms

In [14]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty, 1.2),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [15]:
sims = dict()

In [16]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.6969423294067383
('LLF', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 1.4406752586364746
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 17.976503133773804
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.5220913887023926
('LLF', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.2515177726745605
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 18.447065830230713
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.6092197895050049
('LLF', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 1.2177391052246094
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 18.124445915222168
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.5626301765441895
('LLF', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 1.168686866760254
('MPC', 75)
6

In [17]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [18]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,165.000,35.268,361.684,99.000,0.225
1,three_phase,LLF,60,73.259,72.0,79.208,39.608,264.965,47.525,0.388
2,three_phase,MPC,60,67.432,32.0,95.458,35.602,243.890,57.275,0.362
3,three_phase,Unctrl,65,100.000,100.0,152.308,35.268,361.684,99.000,0.317
4,three_phase,LLF,65,73.144,60.0,92.269,41.067,264.549,59.975,0.366
5,three_phase,MPC,65,68.601,32.0,96.415,36.816,248.117,62.670,0.365
6,three_phase,Unctrl,70,100.000,100.0,141.429,35.268,361.684,99.000,0.309
7,three_phase,LLF,70,73.732,68.0,74.714,40.547,266.676,52.300,0.266
8,three_phase,MPC,70,69.315,32.0,93.386,37.892,250.702,65.370,0.339
9,three_phase,Unctrl,75,100.000,100.0,132.000,35.268,361.684,99.000,0.310
